## This decision tree api is implemented for binary splits with continous and categorical datasets

# Import Statements

In [1]:
import numpy as np
import pandas as pd
from pprint import pprint
import math

# Load and Prepare Data

#### Format of the data
- last column of the data frame must contain the label and it must also be called "label"
- there should be no missing values in the data frame

In [26]:
from sklearn import datasets
import pandas as pd

iris = datasets.load_iris()
df = pd.DataFrame(iris.data)
df.columns = ["sl", "sw", 'pl', 'pw']
df['label'] = iris.target
df['label'].replace({0: 'Iris-setosa', 1: 'Iris-versicolor', 2: 'Iris-virginica'}, inplace = True)


In [3]:
df.head()

,sl,sw,pl,pw,label
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


# Helper Functions

In [4]:
data = df.values
data[:5]

array([[5.1, 3.5, 1.4, 0.2, 'Iris-setosa'],
       [4.9, 3.0, 1.4, 0.2, 'Iris-setosa'],
       [4.7, 3.2, 1.3, 0.2, 'Iris-setosa'],
       [4.6, 3.1, 1.5, 0.2, 'Iris-setosa'],
       [5.0, 3.6, 1.4, 0.2, 'Iris-setosa']], dtype=object)

### Data pure

In [5]:
def check_purity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

### Classify

In [6]:
def classify_data(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

### Find Class counts in "label" column

In [7]:
def calculate_counts_of_classes(data):
    label_column = data[:, -1]
    class_counts = np.array(np.unique(label_column, return_counts=True)).T
    return class_counts

### Potential splits

In [8]:
def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1): # excluding the last column which is the label
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        potential_splits[column_index] = unique_values
    
    return potential_splits

### Split Data

In [15]:
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous":
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values >  split_value]
    
    # feature is categorical   
    else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]
    
    return data_below, data_above

### Lowest Overall Entropy

In [14]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [16]:
def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [12]:
def calculate_split_info(data_below, data_above):
    if(len(data_below) == 0 or len(data_above) == 0):
        return 0
        
    n = len(data_below) + len(data_above)
    si_data_below = len(data_below) / n
    si_data_above = len(data_above) / n
    
    overall_spilt_info =  (si_data_below * -math.log(si_data_below, 2) 
                      + si_data_above * -math.log(si_data_above, 2))
    
    return overall_spilt_info

In [17]:
def determine_best_split(data, potential_splits):
    entropy_before_split = calculate_entropy(data)
    max_gain_ratio = float("-inf")
    
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)
            
            information_gain = entropy_before_split - current_overall_entropy
            split_info = calculate_split_info(data_below, data_above)
            current_gain_ratio = information_gain/split_info if split_info else 0
            
            if  current_gain_ratio >= max_gain_ratio:
                max_gain_ratio = current_gain_ratio
                best_split_column = column_index
                best_split_value = value
                
    return best_split_column, best_split_value, max_gain_ratio, entropy_before_split

### Determine Type of Feature

In [18]:
def determine_type_of_feature(df):
    
    feature_types = []
    n_unique_values_treshold = 15
    for feature in df.columns:
        if feature != "label":
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types

# Decision Tree Algorithm

### Representation of the Decision Tree

### Algorithm

In [19]:
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        node = { 
            "entropy": calculate_entropy(data),
            "level": counter, 
            'class_counts': calculate_counts_of_classes(data) 
        }  
        tree = {'question': classification, 'node': node, 'sub_tree': [] }
        
        return tree

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value, gain_ratio, entropy = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # check for empty data
        if len(data_below) == 0 or len(data_above) == 0:
            classification = classify_data(data)
            node = { 
                "entropy": entropy, 
                "level": counter-1, 
                'class_counts': calculate_counts_of_classes(data) 
            }  
            tree = {'question': classification, 'node': node, 'sub_tree': [] }
            return tree
        
        # determine question
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "continuous":
            question = "{} <= {}".format(feature_name, split_value)
            
        # feature is categorical
        else:
            question = "{} = {}".format(feature_name, split_value)
           
        # instantiate tree
        tree = {'question': question, 'node': {}, 'sub_tree': [] }
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        node = { 
            "entropy": entropy,
            "gain_ratio": gain_ratio,
            "level": counter-1, 
            'class_counts': calculate_counts_of_classes(data) 
        }  
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            tree['question'] = yes_answer
            tree['node'] = node
            tree['sub_tree'] = []
        else:
            tree['sub_tree'].append(yes_answer)
            tree['sub_tree'].append(no_answer)
            tree['question'] = question
            tree['node'] = node
        
        return tree

In [44]:
# example of how a tree looks like
tree = decision_tree_algorithm(df, max_depth = 3)
pprint(tree)

{'node': {'class_counts': array([['Iris-setosa', 50],
       ['Iris-versicolor', 50],
       ['Iris-virginica', 50]], dtype=object),
          'entropy': 1.584962500721156,
          'gain_ratio': 0.9999999999999999,
          'level': 0},
 'question': 'pw <= 0.6',
 'sub_tree': [{'node': {'class_counts': array([['Iris-setosa', 50]], dtype=object),
                        'entropy': 0.0,
                        'level': 1},
               'question': 'Iris-setosa',
               'sub_tree': []},
              {'node': {'class_counts': array([['Iris-versicolor', 50],
       ['Iris-virginica', 50]], dtype=object),
                        'entropy': 1.0,
                        'gain_ratio': 0.6933647985912662,
                        'level': 1},
               'question': 'pw <= 1.7',
               'sub_tree': [{'node': {'class_counts': array([['Iris-versicolor', 49],
       ['Iris-virginica', 5]], dtype=object),
                                      'entropy': 0.44506485705083865,
   

# Classification

In [21]:
example = df.iloc[0]
example

sl               5.1
sw               3.5
pl               1.4
pw               0.2
label    Iris-setosa
Name: 0, dtype: object

In [22]:
def classify_example(example, tree):
    q = tree['question']
    # base case
    if(len(q.split(" ")) != 3):
        return q

    feature_name, comparison_operator, value = q.split(" ")

    # ask question
    if comparison_operator == "<=":
        if example[feature_name] <= float(value):
            answer = tree['sub_tree'][0]
        else:
            answer = tree['sub_tree'][1]
    
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree['sub_tree'][0]
        else:
            answer = tree['sub_tree'][1]

    # recursive part
    residual_tree = answer
    return classify_example(example, residual_tree)
        

In [23]:
classify_example(example, tree)


'Iris-setosa'

# Calculate Accuracy

In [24]:
def calculate_accuracy(df, tree):
    new_df = pd.DataFrame()
    new_df["classification"] = df.apply(classify_example, args=(tree,), axis=1)
    new_df["classification_correct"] = new_df["classification"] == df["label"]
    
    accuracy = new_df["classification_correct"].mean()
    
    return accuracy

### Train and test split

In [49]:
import random
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

## Iris dataset. 

In [68]:
random.seed(0)
train_df, test_df = train_test_split(df, test_size=20)

tree = decision_tree_algorithm(train_df)
accuracy = calculate_accuracy(test_df, tree)

accuracy

0.9

In [70]:
def print_tree(tree):
    q = tree['question']
    
    # base case
    if(len(q.split(" ")) != 3):
        print("Level :", tree['node']['level'])
        print("Current Entropy :", tree['node']['entropy'])
        for c in tree['node']['class_counts']:
            print("Count of",c[0],":", c[1])
        print("Reached leaf node")
        print("--------------------------")
        return

    feature_name, comparison_operator, value = q.split(" ")
    print("Level :", tree['node']['level'])
    print("Curret Entropy :", tree['node']['entropy'])
    print("Splitting on Feature :", feature_name)
    print("Gain ratio :", tree['node']['gain_ratio'])

    for c in tree['node']['class_counts']:
        print("Count of",c[0],":", c[1])
    print("--------------------------")

    print_tree(tree['sub_tree'][0])
    print_tree(tree['sub_tree'][1])
    return
print_tree(tree)

Level : 0
Curret Entropy : 1.5836100169358511
Splitting on Feature : pw
Gain ratio : 1.0
Count of Iris-setosa : 46
Count of Iris-versicolor : 42
Count of Iris-virginica : 42
--------------------------
Level : 1
Current Entropy : 0.0
Count of Iris-setosa : 46
Reached leaf node
--------------------------
Level : 1
Curret Entropy : 1.0
Splitting on Feature : pw
Gain ratio : 0.7327834928979089
Count of Iris-versicolor : 42
Count of Iris-virginica : 42
--------------------------
Level : 2
Curret Entropy : 0.3591016256485496
Splitting on Feature : pl
Gain ratio : 0.6492628558805021
Count of Iris-versicolor : 41
Count of Iris-virginica : 3
--------------------------
Level : 3
Current Entropy : 0.0
Count of Iris-versicolor : 40
Reached leaf node
--------------------------
Level : 3
Curret Entropy : 0.8112781244591328
Splitting on Feature : pw
Gain ratio : 1.0
Count of Iris-versicolor : 1
Count of Iris-virginica : 3
--------------------------
Level : 4
Current Entropy : 0.0
Count of Iris-virgin

# 1. OR dataset

In [52]:
a = ['True', 'True', 'True']
b = ['False', 'True', 'True']
c = ['True', 'False', 'True']
d = ['False', 'False', 'False']
data = [a,b,c,d]
df1 = pd.DataFrame(data, columns = ['a', 'b', 'label']) 


In [60]:
tree = decision_tree_algorithm(df1)
accuracy = calculate_accuracy(df1, tree)
accuracy

1.0

In [61]:
print_tree(tree)

Level : 0
Curret Entropy : 0.8112781244591328
Splitting on Feature : b
Gain ratio : 0.31127812445913283
Count of False : 1
Count of True : 3
--------------------------
Level : 1
Current Entropy : 0.0
Count of True : 2
Reached leaf node
--------------------------
Level : 1
Curret Entropy : 1.0
Splitting on Feature : a
Gain ratio : 1.0
Count of False : 1
Count of True : 1
--------------------------
Level : 2
Current Entropy : 0.0
Count of True : 1
Reached leaf node
--------------------------
Level : 2
Current Entropy : 0.0
Count of False : 1
Reached leaf node
--------------------------


# 2. Iris dataset Categorical

In [62]:
from sklearn import datasets
import pandas as pd

iris = datasets.load_iris()

df3 = pd.DataFrame(iris.data)
df3.columns = ["sl", "sw", 'pl', 'pw']

#Function to find label for a value
#if MIN_Value <=val < (m + Mean_Value) / 2 then it is assigned label a
#if (m + Mean_Value) <=val < Mean_Value then it is assigned label b
#if (Mean_Value) <=val < (Mean_Value + MAX_Value)/2 then it is assigned label c
#if (Mean_Value + MAX_Value)/2 <=val <= MAX_Value  then it is assigned label d

def label(val, *boundaries):
    if (val < boundaries[0]):
        return 'a'
    elif (val < boundaries[1]):
        return 'b'
    elif (val < boundaries[2]):
        return 'c'
    else:
        return 'd'

#Function to convert a continuous data into labelled data
#There are 4 lables  - a, b, c, d
def toLabel(df3, old_feature_name):
    second = df3[old_feature_name].mean()
    minimum = df3[old_feature_name].min()
    first = (minimum + second)/2
    maximum = df3[old_feature_name].max()
    third = (maximum + second)/2
    return df3[old_feature_name].apply(label, args= (first, second, third))

df3['sl'] = toLabel(df3, 'sl')
df3['sw'] = toLabel(df3, 'sw')
df3['pl'] = toLabel(df3, 'pl')
df3['pw'] = toLabel(df3, 'pw')
df3['label'] = iris.target
df3['label'].replace({0: 'Iris-setosa', 1: 'Iris-versicolor', 2: 'Iris-virginica'}, inplace = True)

In [64]:
random.seed(0)
train_df3, test_df3 = train_test_split(df3, test_size=20)

tree = decision_tree_algorithm(train_df3)
accuracy = calculate_accuracy(test_df3, tree)

accuracy

0.95

In [65]:
print_tree(tree)

Level : 0
Curret Entropy : 1.5836100169358511
Splitting on Feature : pw
Gain ratio : 1.0
Count of Iris-setosa : 46
Count of Iris-versicolor : 42
Count of Iris-virginica : 42
--------------------------
Level : 1
Current Entropy : 0.0
Count of Iris-setosa : 46
Reached leaf node
--------------------------
Level : 1
Curret Entropy : 1.0
Splitting on Feature : pw
Gain ratio : 0.5410329375826775
Count of Iris-versicolor : 42
Count of Iris-virginica : 42
--------------------------
Level : 2
Current Entropy : 0.0
Count of Iris-virginica : 30
Reached leaf node
--------------------------
Level : 2
Curret Entropy : 0.7642045065086203
Splitting on Feature : pl
Gain ratio : 0.510748581606675
Count of Iris-versicolor : 42
Count of Iris-virginica : 12
--------------------------
Level : 3
Current Entropy : 0.0
Count of Iris-virginica : 5
Reached leaf node
--------------------------
Level : 3
Curret Entropy : 0.5916727785823275
Splitting on Feature : sl
Gain ratio : 0.09830480375510302
Count of Iris-ve